In [1]:
# Import csv records of both datasets
import pandas as pd

# Load CSV file
df1 = pd.read_csv("final_unique_records.csv") 
df2 = pd.read_csv("TrueCars_data.csv") 

In [2]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45536 entries, 0 to 45535
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   bodyStyle                         45536 non-null  object 
 1   color                             45536 non-null  object 
 2   drivingMiles                      44161 non-null  float64
 3   earliestLocalLocation_locationId  44161 non-null  float64
 4   fuelType                          45536 non-null  object 
 5   interiorColor                     45536 non-null  object 
 6   kbbTrim                           45536 non-null  object 
 7   locationId                        45536 non-null  int64  
 8   make                              45536 non-null  object 
 9   mileage                           45536 non-null  int64  
 10  milesPerGallon                    45536 non-null  int64  
 11  model                             45536 non-null  object 
 12  pare

In [3]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45920 entries, 0 to 45919
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Car Name                   45920 non-null  object 
 1   Car Webpage                45920 non-null  object 
 2   Car Health                 45920 non-null  object 
 3   Price                      45894 non-null  object 
 4   Car Exterior Color         45920 non-null  object 
 5   Car Interior Color         45920 non-null  object 
 6   Miles                      45920 non-null  object 
 7   Fuel Type                  45894 non-null  object 
 8   Fuel Efficiency (mileage)  41323 non-null  object 
 9   EPA Range                  2005 non-null   object 
 10  Transmission               45894 non-null  object 
 11  Drivetrain                 45894 non-null  object 
 12  Engine                     45920 non-null  object 
 13  Location                   45920 non-null  obj

In [272]:
# Make column names same in all. So check column names in both for renaming

print("df1 columns:", df1.columns)
print("df2 columns:", df2.columns)

df1 columns: Index(['bodyStyle', 'color', 'drivingMiles',
       'earliestLocalLocation_locationId', 'fuelType', 'interiorColor',
       'kbbTrim', 'locationId', 'make', 'mileage', 'milesPerGallon', 'model',
       'parentModel', 'price_kbbValue', 'price_total', 'seatingCapacity',
       'stockNumber', 'trim', 'vdpSlug', 'vehicleId', 'vehicleInventoryType',
       'vin', 'year', 'zip_code'],
      dtype='object')
df2 columns: Index(['Car Name', 'Car Webpage', 'Car Health', 'Price', 'Car Exterior Color',
       'Car Interior Color', 'Miles', 'Fuel Type', 'Fuel Efficiency (mileage)',
       'EPA Range', 'Transmission', 'Drivetrain', 'Engine', 'Location',
       'Listing Status', 'VIN', 'Stock Number', 'Accidents', 'Owners',
       'Car Title Status', 'Use Type', 'Vehicle Last Inspected', 'Make',
       'Model', 'Trim', 'Body Class', 'Model Year', 'Seating Capacity'],
      dtype='object')


In [273]:
# Both datasets have location information in different formats. So we have used Nominatim API to extract location information for both datasets. 
# First, we observed that 95213 and 02704 zipcodes are fetching wrong location information from the API. So we replaced them with 95212 and 02760 which are neighboring zip codes.

df1['zip_code'] = df1['zip_code'].replace({'02704': '02760', '95213': '95212'})

In [274]:
#Use Nominatim API to extract location details for Carvana dataset with zip code as the query
import pandas as pd
import requests
import time

# Duplicate df1 before modifying
df_cv = df1.copy()

# Ensure zip_code is stored as a string and formatted to 5 digits
df_cv['zip_code'] = df_cv['zip_code'].astype(str).str.zfill(5)

# Function to fetch location details using Nominatim
def get_zip_info(zip_code):
    """Fetch city, state, latitude, and longitude for a given ZIP code."""
    zip_code = str(zip_code).zfill(5)  # Ensure 5-digit format by padding with 0 if needed

    url = "https://nominatim.openstreetmap.org/search?"
    params = {
        'email': 'xyz@gmail.com',  # Replace with your email for compliance
        "q": f"{zip_code}, USA",
        "format": "json",
        "addressdetails": "1",
        "limit": "1"
    }

    start_time = time.time()
    try:
        response = requests.get(url, params=params)
        request_time = time.time() - start_time

        if response.status_code == 200 and response.json():
            data = response.json()[0]
            place = data.get('address', {}).get('city', None) or data.get('address', {}).get('town', None) or data.get('address', {}).get('village', None)
            state = data.get('address', {}).get('state', None)
            lat = data.get('lat', None)
            lon = data.get('lon', None)
            return {"place": place, "state": state, "latitude": lat, "longitude": lon, "request_time": request_time}
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {zip_code}: {e}")

    return {"place": None, "state": None, "latitude": None, "longitude": None, "request_time": request_time}

# Extract unique ZIP codes in correct format
unique_zip_codes = df_cv['zip_code'].dropna().unique()

# Fetch data only for unique ZIP codes
zip_data = {}
total_start = time.time()
times = []

for idx, zip_code in enumerate(unique_zip_codes):
    result = get_zip_info(zip_code)
    zip_data[zip_code] = result
    times.append(result["request_time"])

    # Print estimated time remaining every 10 ZIP codes
    if (idx + 1) % 10 == 0 or (idx + 1) == len(unique_zip_codes):
        avg_time = sum(times) / len(times) if times else 0
        remaining = avg_time * (len(unique_zip_codes) - idx - 1)
        print(f"Processed {idx+1}/{len(unique_zip_codes)} unique ZIP codes. Avg time: {avg_time:.2f}s/request. Estimated time left: {remaining:.2f}s")

# Map results back to the original dataframe
df_cv['place'] = df_cv['zip_code'].map(lambda x: zip_data.get(x, {}).get('place'))
df_cv['state'] = df_cv['zip_code'].map(lambda x: zip_data.get(x, {}).get('state'))
df_cv['latitude'] = df_cv['zip_code'].map(lambda x: zip_data.get(x, {}).get('latitude'))
df_cv['longitude'] = df_cv['zip_code'].map(lambda x: zip_data.get(x, {}).get('longitude'))

# Ensure ZIP codes remain as strings before saving
df_cv['zip_code'] = df_cv['zip_code'].astype(str).str.zfill(5)

# Save to CSV
df_cv.to_csv("zip_details_carvana.csv", index=False)
df1['zip_code'] = df1['zip_code'].astype(str).str.zfill(5)  # Apply the fix to df1 as well
df1.to_csv("zip_original_carvana.csv", index=False)

# Total execution time
total_time = time.time() - total_start
print(f"\nTotal execution time: {total_time:.2f} seconds ({total_time / 60:.2f} minutes)")

Processed 10/60 unique ZIP codes. Avg time: 0.96s/request. Estimated time left: 48.22s
Processed 20/60 unique ZIP codes. Avg time: 1.06s/request. Estimated time left: 42.22s
Processed 30/60 unique ZIP codes. Avg time: 1.01s/request. Estimated time left: 30.30s
Processed 40/60 unique ZIP codes. Avg time: 1.01s/request. Estimated time left: 20.14s
Processed 50/60 unique ZIP codes. Avg time: 1.01s/request. Estimated time left: 10.08s
Processed 60/60 unique ZIP codes. Avg time: 1.01s/request. Estimated time left: 0.00s

Total execution time: 60.88 seconds (1.01 minutes)


In [276]:
df_cv.head(5)

,bodyStyle,color,drivingMiles,earliestLocalLocation_locationId,fuelType,interiorColor,kbbTrim,locationId,make,mileage,...,vdpSlug,vehicleId,vehicleInventoryType,vin,year,zip_code,place,state,latitude,longitude
0,Convertible,Black,359.0,341.0,Gas,Black,SL 550 Roadster 2D,111,Mercedes-Benz,44684,...,2012-mercedes-benz-sl-class-sl-550,3276672,1,WDBSK7BA7CF168963,2012,32218,Jacksonville,Florida,30.46844200107991,-81.64994234298057
1,Pickup,Gray,180.0,341.0,Gas,Black,Big Horn Pickup 4D 5 1/2 ft,128,Ram,58318,...,2018-ram-1500-crew-cab-big-horn-5-12-ft,3436816,1,1C6RR6LT6JS180829,2018,32218,Jacksonville,Florida,30.46844200107991,-81.64994234298057
2,Pickup,Gray,180.0,341.0,Gas,Black,Texas Trail Edition Pickup 4D 5 ft,128,Jeep,40407,...,2022-jeep-gladiator-texas-trail-edition-5-ft,3425231,1,1C6HJTAG5NL164399,2022,32218,Jacksonville,Florida,30.46844200107991,-81.64994234298057
3,Sedan,Gray,180.0,341.0,Gas,Black,1.4T S Sedan 4D,128,Volkswagen,68830,...,2021-volkswagen-jetta-1.4t-s,3493604,1,3VWC57BU6MM089869,2021,32218,Jacksonville,Florida,30.46844200107991,-81.64994234298057
4,Suv,Black,180.0,341.0,Gas,Black,xDrive40i Sport Utility 4D,128,BMW,68880,...,2019-bmw-x5-xdrive40i,3366116,1,5UXCR6C51KLK80556,2019,32218,Jacksonville,Florida,30.46844200107991,-81.64994234298057


In [278]:
#Use Nominatim API to extract location details for TrueCar dataset with zip code as the query
import pandas as pd
import requests
import time

# Duplicate df2 before modifying
df_tc = df2.copy()

# Function to fetch ZIP code, latitude, and longitude using Nominatim
def get_location_info(city, state):
    """Fetch ZIP code, latitude, and longitude for a given city, state."""
    url = "https://nominatim.openstreetmap.org/search?"
    params = {
        'email': 'xyz@gmail.com',  # Replace with your email for compliance
        "q": f"{city}, {state}, USA",
        "format": "json",
        "addressdetails": "1",
        "limit": "1"
    }
    
    start_time = time.time()
    try:
        response = requests.get(url, params=params)
        request_time = time.time() - start_time

        if response.status_code == 200 and response.json():
            data = response.json()[0]
            return {
                "zip_code": data.get('address', {}).get('postcode', None),
                "latitude": data.get('lat', None),
                "longitude": data.get('lon', None),
                "request_time": request_time,
                "place": data.get('address', {}).get('city', None),  # City (place)
                "state": data.get('address', {}).get('state', None)  # State
            }
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {city}, {state}: {e}")
    
    return {"zip_code": None, "latitude": None, "longitude": None, "request_time": request_time, "place": None, "state": None}

# Correction dictionary for known location errors
location_corrections = {
    "Liburn, GA (1962 miles away)": "Lilburn, GA",
    "St. Peters, MO (1579 miles away)": "St. Peters, MO",
    "Altanta, GA (1947 miles away)": "Atlanta, GA",
    "Fort Lauderdale,, FL (2340 miles away)": "Fort Lauderdale, FL",
    "Sacramennto, CA (359 miles away)": "Sacramento, CA",
    "Santa Clarita,, CA (28 miles away)": "Santa Clarita, CA",
    "Husbrouck Heights, NJ (2456 miles away)": "Hasbrouck Heights, NJ",
    "Wesy Babylon, NY (2495 miles away)": "West Babylon, NY",
    "Online Retailer": None,  # Not a valid location
    "Panorma City, CA (16 miles away)": "Panorama City, CA",
    "Fort Lauderdale,, FL (460 miles away)": "Fort Lauderdale, FL",
    "Brandenton, FL (402 miles away)": "Bradenton, FL",
    "Hasbrouk Heights, NJ (645 miles away)": "Hasbrouck Heights, NJ",
    "Cincinnnati, OH (514 miles away)": "Cincinnati, OH"
}

# Extract city-state pairs and apply corrections
df_tc['city_state'] = df_tc['Location'].replace(location_corrections)
df_tc['city_state'] = df_tc['city_state'].str.extract(r'([\w\s-]+),\s([A-Z]{2})').apply(lambda x: ', '.join(x.dropna()), axis=1)

# Drop NaN values and ensure unique locations
unique_locations = df_tc['city_state'].dropna().unique()

# Fetch data for unique locations
location_data = {}
total_start = time.time()
times = []

for idx, loc in enumerate(unique_locations):
    parts = loc.split(', ')
    
    # Ensure there are exactly two parts (city, state)
    if len(parts) == 2:
        city, state = parts
        result = get_location_info(city, state)
    else:
        print(f"Skipping invalid location format: {loc}")
        result = {"zip_code": None, "latitude": None, "longitude": None, "request_time": None, "place": None, "state": None}
    
    location_data[loc] = result
    times.append(result["request_time"] if result["request_time"] else 0)

    # Print estimated time remaining every 10 locations
    if (idx + 1) % 100 == 0 or (idx + 1) == len(unique_locations):
        avg_time = sum(times) / len(times) if times else 0
        remaining = avg_time * (len(unique_locations) - idx - 1)
        print(f"Processed {idx+1}/{len(unique_locations)} unique locations. Avg time: {avg_time:.2f}s/request. Estimated time left: {remaining:.2f}s")

# Map results back to original dataframe
df_tc['zip_code'] = df_tc['city_state'].map(lambda x: location_data.get(x, {}).get('zip_code'))
df_tc['latitude'] = df_tc['city_state'].map(lambda x: location_data.get(x, {}).get('latitude'))
df_tc['longitude'] = df_tc['city_state'].map(lambda x: location_data.get(x, {}).get('longitude'))
df_tc['place'] = df_tc['city_state'].map(lambda x: location_data.get(x, {}).get('place'))  # Add place (city)
df_tc['state'] = df_tc['city_state'].map(lambda x: location_data.get(x, {}).get('state'))  # Add state

# Drop helper column
df_tc.drop(columns=['city_state'], inplace=True)

# Save to CSV
df_tc.to_csv("location_details_tc.csv", index=False)

# Total execution time
total_time = time.time() - total_start
print(f"\nTotal execution time: {total_time:.2f} seconds ({total_time / 60:.2f} minutes)")

Processed 100/1113 unique locations. Avg time: 1.00s/request. Estimated time left: 1012.53s
Processed 200/1113 unique locations. Avg time: 1.02s/request. Estimated time left: 932.14s
Processed 300/1113 unique locations. Avg time: 1.03s/request. Estimated time left: 835.96s
Processed 400/1113 unique locations. Avg time: 1.02s/request. Estimated time left: 730.13s
Processed 500/1113 unique locations. Avg time: 1.02s/request. Estimated time left: 626.41s
Processed 600/1113 unique locations. Avg time: 1.02s/request. Estimated time left: 522.72s
Skipping invalid location format: 
Processed 700/1113 unique locations. Avg time: 1.02s/request. Estimated time left: 420.18s
Processed 800/1113 unique locations. Avg time: 1.02s/request. Estimated time left: 318.37s
Processed 900/1113 unique locations. Avg time: 1.02s/request. Estimated time left: 216.24s
Processed 1000/1113 unique locations. Avg time: 1.01s/request. Estimated time left: 114.59s
Processed 1100/1113 unique locations. Avg time: 1.01s

In [119]:
#df_tc.head()

In [279]:
#Create copies of updated dataframes to avoide accidental overwrite
df_cv1 = df_cv.copy()
df_tc1 = df_tc.copy()

In [280]:
# Remove redundant columns from both datasets.
df_tc_colremove = ['Car Name', 'Car Webpage','Car Exterior Color','Car Interior Color', 'Stock Number','Car Title Status','Vehicle Last Inspected', 'Use Type','Seating Capacity', 'city','Location']
df_cv_colremove = ['color','earliestLocalLocation_locationId','interiorColor','kbbTrim','locationId','parentModel','stockNumber','vdpSlug','vehicleId','vehicleInventoryType','Accidents',
    'Owners', 'drivingMiles','Location', 'seatingCapacity']

columns_to_remove = ['col1', 'col2', 'col3']
df_tc2 = df_tc1.drop(columns=[col for col in df_tc_colremove if col in df_tc1.columns])
df_cv2 = df_cv1.drop(columns=[col for col in df_cv_colremove if col in df_cv1.columns])
              # Car title is clean for all

In [141]:
#df_tc2.columns

In [140]:
#df_cv2.columns

In [281]:
# Rename variables in tc2 to match cv2
column_mapping = {   
    'Price':'price_total',
    'Miles':'mileage',
    'Fuel Type':'fuelType',
    'Fuel Efficiency (mileage)': 'MPG', # This will later be used for extracting city and highway mpg
    'EPA Range':'range',
    'Transmission':'transmission',
    'Listing Status' : 'listingStatus', # this is not present in cv dataset
    'VIN':'vin',
    'Make':'make',
    'Model':'model',
    'Trim':'trim',
    'Body Class':'bodyStyle',
    'Model Year':'year',
    'Drivetrain':'drivetrain',
    'Engine':'engine',
    'Accidents':'accidents',
    'Car Health':'carHealth',
    #'MPG':'milesPerGallon',
    'Owners':'owners'
}
df_tc2.rename(columns=column_mapping, inplace=True)

In [156]:
#print(sorted(df_tc2.columns))

In [144]:
#print( sorted(df_cv2.columns))

In [145]:
#check = df_tc2[df_tc2['zip_code'].isnull()]
#check = df_cv2[df_cv2['zip_code'].isnull()]
#(check)
#no missing latitude and state information

In [282]:
#Most of the variables in TrueCar dataset will require cleaning to store it in appropriate formats.
import pandas as pd
import numpy as np
import re

# Function to clean price
def clean_price(value):
    if isinstance(value, str):
        return int(value.replace("$", "").replace(",", "").strip())
    return np.nan

# Function to clean mileage
def clean_mileage(value):
    if isinstance(value, str):
        return int(value.replace("miles", "").replace(",", "").strip())
    return np.nan

# Function to extract city MPG, highway MPG, and calculate average MPG
def extract_mpg(value):
    if isinstance(value, str) and " city / " in value:
        try:
            city, highway = value.split(" city / ")
            city_mpg = float(city.strip())
            highway_mpg = float(highway.replace(" highway", "").strip())
            avg_mpg = (city_mpg + highway_mpg) / 2
            return city_mpg, highway_mpg, avg_mpg
        except ValueError:
            print(f"Error parsing MPG value: {value}")
    elif isinstance(value, str) and "city" in value.lower() and "highway" in value.lower():
        print(f"Unexpected MPG format: {value}")
    return np.nan, np.nan, np.nan

# Function to clean range
def clean_range(value):
    if isinstance(value, str):
        return float(value.replace("mi", "").strip())
    return np.nan

# Function to clean zip_code (convert to string, ensure 5-digit format)
def clean_zip(value):
    if pd.notna(value):
        return str(value).zfill(5)
    return None

# Function to clean listingStatus and extract days
def extract_days(value):
    if isinstance(value, str):
        match = re.search(r"(\d+)\s+days?", value)  # Look for a number followed by 'days'
        if match:
            return int(match.group(1))
    return np.nan

# Function to clean accidents column
def clean_accidents(value):
    if isinstance(value, str):
        match = re.search(r"(\d+)", value)  # Extracts the first number found
        if match:
            return int(match.group(1))
    return np.nan  # Return NaN if no number is found

# Function to clean engine size (extract numbers before "L")
def clean_engine(value):
    if isinstance(value, str):
        match = re.search(r"([\d.]+)L", value)  # Extracts numbers before "L"
        if match:
            return float(match.group(1))  # Convert to float
    return np.nan

# Apply transformations to df_tc2
df_tc2["price_total"] = df_tc2["price_total"].apply(clean_price)
df_tc2["mileage"] = df_tc2["mileage"].apply(clean_mileage)
df_tc2["accidents"] = df_tc2["accidents"].apply(clean_accidents)

# Extract city MPG, highway MPG, and calculate average MPG from MPG column
if "MPG" in df_tc2.columns:
    mpg_data = df_tc2["MPG"].apply(lambda x: pd.Series(extract_mpg(x), index=["mpg_city", "mpg_highway", "milesPerGallon"]))
    df_tc2 = df_tc2.join(mpg_data)

df_tc2["range"] = df_tc2["range"].apply(clean_range)
df_tc2["zip_code"] = df_tc2["zip_code"].apply(clean_zip)
df_tc2["latitude"] = pd.to_numeric(df_tc2["latitude"], errors="coerce")
df_tc2["longitude"] = pd.to_numeric(df_tc2["longitude"], errors="coerce")
df_tc2["listingDays"] = df_tc2["listingStatus"].apply(extract_days)
df_tc2["engine_size"] = df_tc2["engine"].apply(clean_engine)  # Clean engine size

# Handle 'place' and 'state' columns from df_tc2 or df_cv2 and add any missing info
df_tc2["place"] = df_tc2["place"].apply(lambda x: x if isinstance(x, str) else None)
df_tc2["state"] = df_tc2["state"].apply(lambda x: x if isinstance(x, str) else None)

# Drop unnecessary columns
df_tc2.drop(columns=["listingStatus", "mpg_city", "mpg_highway"], inplace=True, errors="ignore")

# Print updated DataFrame info
print(df_tc2.dtypes)

carHealth          object
price_total       float64
mileage             int64
fuelType           object
MPG                object
range             float64
transmission       object
drivetrain         object
engine             object
vin                object
accidents           int64
owners            float64
make               object
model              object
trim               object
bodyStyle          object
year                int64
zip_code           object
latitude          float64
longitude         float64
place              object
state              object
milesPerGallon    float64
listingDays         int64
engine_size       float64
dtype: object


In [283]:
df_tc2.head()

,carHealth,price_total,mileage,fuelType,MPG,range,transmission,drivetrain,engine,vin,...,bodyStyle,year,zip_code,latitude,longitude,place,state,milesPerGallon,listingDays,engine_size
0,Used,18300.0,50691,Gas,22 city / 27 highway,NaN,Automatic Transmission,AWD,2.0L Inline-4 Gas Inline,3GNKBHR40PS106689,...,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,2023,None,28.811735,-81.268022,Sanford,Florida,24.5,86,2.0
1,Used,40998.0,12745,Gas,15 city / 24 highway,NaN,Manual Transmission,RWD,5.0L V-8 Gas V,1FA6P8K03K5504880,...,Coupe,2019,None,33.350293,-86.851123,None,Alabama,19.5,4,5.0
2,Used,35470.0,27851,Gas,19 city / 22 highway,NaN,Automatic Transmission,4WD,5.0L V-8 Gas V,1FTFW1E54MKD35226,...,Pickup,2021,30144,34.023434,-84.615490,None,Georgia,20.5,87,5.0
3,Certified Pre-Owned,35525.0,30957,Gas,20 city / 22 highway,NaN,Automatic Transmission,4WD,2.3L Inline-4 Gas Inline,1FMDE5BH0MLA83903,...,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,2021,None,32.840695,-83.632402,Macon,Georgia,21.0,3,2.3
4,Used,11900.0,86024,Gas,23 city / 31 highway,NaN,Automatic Transmission,AWD,2.0L Inline-4 Gas Inline,WDCTG4GB4HJ314998,...,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,2017,None,26.461462,-80.072820,None,Florida,27.0,24,2.0


In [284]:
# Bring all carvana data variables to right format
import pandas as pd
import numpy as np

# Function to clean mileage
def clean_mileage(value):
    if isinstance(value, str):
        return int(value.replace("miles", "").replace(",", "").strip())
    return np.nan

# Function to clean zip_code (convert to string, ensure 5-digit format)
def clean_zip(value):
    if pd.notna(value):
        return str(value).zfill(5)
    return None

# Function to convert make and model to upper case
def clean_make_model(df):
    df["make"] = df["make"].str.upper()
    df["model"] = df["model"].str.upper()
    return df

# Convert data types
df_cv2["mileage"] = pd.to_numeric(df_cv2["mileage"], errors="coerce")
df_cv2["milesPerGallon"] = pd.to_numeric(df_cv2["milesPerGallon"], errors="coerce")
df_cv2["price_kbbValue"] = pd.to_numeric(df_cv2["price_kbbValue"], errors="coerce")
df_cv2["price_total"] = pd.to_numeric(df_cv2["price_total"], errors="coerce")
df_cv2["zip_code"] = df_cv2["zip_code"].apply(clean_zip)
df_cv2["latitude"] = pd.to_numeric(df_cv2["latitude"], errors="coerce")
df_cv2["longitude"] = pd.to_numeric(df_cv2["longitude"], errors="coerce")

# Apply upper case function to make and model
df_cv2 = clean_make_model(df_cv2)

In [285]:
# KBB values are missing in the TrueCar dataset. So impute them using regression.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Make copies of df_cv2 and df_tc2
df_cv3 = df_cv2.copy()
df_tc3 = df_tc2.copy()

# Select numerical and categorical features
num_features = ["mileage", "milesPerGallon", "year", "price_total"]
cat_features = ["fuelType", "state"]

# Drop rows where price_kbbValue is missing in df_cv3
df_cv3 = df_cv3.dropna(subset=["price_kbbValue"])

# Apply log transformation to the target variable
df_cv3["log_price_kbbValue"] = np.log(df_cv3["price_kbbValue"])

# Split features (X) and target (y) for df_cv3
X = df_cv3[num_features + cat_features]
y = df_cv3["log_price_kbbValue"]  # Now using log-transformed target

# Define preprocessing pipeline
num_transformer = SimpleImputer(strategy="median")
cat_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", num_transformer, num_features),
    ("cat", cat_transformer, cat_features)
])

# Define model pipeline
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model.fit(X_train, y_train)

# Predict on test set and evaluate (optional)
y_pred_log = model.predict(X_test)
y_pred = np.exp(y_pred_log)  # Convert back to original scale

print(f"Test R² Score: {model.score(X_test, y_test):.4f}")

# Prepare df_tc3 for prediction (ensure same columns as X)
X_tc3 = df_tc3[num_features + cat_features]

# Predict prices for df_tc3 (convert back from log)
df_tc3["price_kbbValue"] = np.exp(model.predict(X_tc3))

# Save results for df_tc3
df_tc3.to_csv("tc3_with_predicted_prices.csv", index=False)

print("Predictions saved successfully for df_tc3!")

Test R² Score: 0.9688
Predictions saved successfully for df_tc3!


In [296]:

# Descriptive statistics of predicted values
predicted_stats = df_tc3["price_kbbValue"].describe()

# Print the statistics
print(predicted_stats)

count     45920.000000
mean      24694.271829
std       16025.952218
min        5726.735185
25%       13282.054729
50%       21930.506476
75%       31517.486501
max      105621.626798
Name: price_kbbValue, dtype: float64


In [297]:
# Add identifiers to both columns to differentiate their sources.
df_cv3["source"] = "CV" #Carvana
df_tc3["source"] = "TC" #TrueCar

In [298]:
#Combine carvana and truecar data frames
import pandas as pd

# Combine both DataFrames, keeping all columns and filling missing values with NaN
df_combined = pd.concat([df_tc3, df_cv3], axis=0, ignore_index=True)


In [299]:
df_combined["bodyStyle"].unique()
#too many categories. reduce them

array(['Sport Utility Vehicle (SUV)/Multi-Purpose Vehicle (MPV)', 'Coupe',
       'Pickup', 'Hatchback/Liftback/Notchback', 'Sedan/Saloon',
       'Convertible/Cabriolet', 'Minivan',
       'Crossover Utility Vehicle (CUV)', 'Wagon', 'Van', 'Cargo Van',
       'Roadster', 'Sport Utility Truck (SUT)', 'Incomplete - Cutaway',
       'Incomplete - Chassis Cab (Single Cab)',
       'Incomplete - Chassis Cab (Double Cab)', nan,
       'Incomplete - Chassis Cab (Number of Cab Unknown)', 'Truck',
       'Incomplete - Stripped Chassis', 'Incomplete', 'Limousine',
       'Convertible', 'Sedan', 'Suv', 'MiniVan', 'Hatchback'],
      dtype=object)

In [300]:
# Function to categorize bodyStyle
def categorize_body_style(value):
    if isinstance(value, str):
        value = value.lower()  # Normalize case
        if "suv" in value or "mpv" in value or "cuv" in value:
            return "SUV/CUV"
        elif "sedan" in value or "saloon" in value:
            return "Sedan"
        elif "hatchback" in value or "liftback" in value or "notchback" in value:
            return "Hatchback"
        elif "coupe" in value or "convertible" in value or "cabriolet" in value or "roadster" in value:
            return "Coupe/Convertible"
        elif "pickup" in value or "sut" in value or "truck" in value:
            return "Pickup/Truck"
        elif "minivan" in value or "van" in value or "cargo van" in value:
            return "Van/Minivan"
        elif "chassis" in value or "cutaway" in value or "incomplete" in value:
            return "Chassis/Incomplete Vehicles"
        elif "wagon" in value:
            return "Wagon"
        elif "limousine" in value:
            return "Limousine"
    return "Other"  # Default category for unexpected values

# Apply categorization to the DataFrame
df_combined["bodyStyleCategory"] = df_combined["bodyStyle"].apply(categorize_body_style)

# Check the results
print(df_combined["bodyStyleCategory"].value_counts())

bodyStyleCategory
SUV/CUV                        45490
Sedan                          23668
Pickup/Truck                   10177
Coupe/Convertible               4197
Hatchback                       4139
Van/Minivan                     2414
Wagon                           1107
Chassis/Incomplete Vehicles      117
Other                             17
Limousine                          1
Name: count, dtype: int64


In [291]:
#Fueltype gas a few values with wrong capitalization in "gas" - changed to Gas
df_combined["fuelType"] = df_combined["fuelType"].replace({"gas": "Gas"})

In [293]:
# Logprices not relevant anymore.
df_combined = df_combined.drop(columns=['log_price_kbbValue'])

In [294]:
# Save to CSV
df_combined.to_csv("combined_data_final v2.csv", index=False)

# Display summary
print(df_combined.info())
print(df_combined.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91327 entries, 0 to 91326
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   carHealth          45920 non-null  object 
 1   price_total        91301 non-null  float64
 2   mileage            91327 non-null  int64  
 3   fuelType           91301 non-null  object 
 4   MPG                41323 non-null  object 
 5   range              2005 non-null   float64
 6   transmission       45894 non-null  object 
 7   drivetrain         45894 non-null  object 
 8   engine             45920 non-null  object 
 9   vin                91327 non-null  object 
 10  accidents          45920 non-null  float64
 11  owners             45918 non-null  float64
 12  make               91325 non-null  object 
 13  model              91311 non-null  object 
 14  trim               76174 non-null  object 
 15  bodyStyle          91310 non-null  object 
 16  year               913

In [237]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91327 entries, 0 to 91326
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   carHealth           45920 non-null  object 
 1   price_total         91301 non-null  float64
 2   mileage             91327 non-null  int64  
 3   fuelType            91301 non-null  object 
 4   MPG                 41323 non-null  object 
 5   range               2005 non-null   float64
 6   transmission        45894 non-null  object 
 7   drivetrain          45894 non-null  object 
 8   engine              45920 non-null  object 
 9   vin                 91327 non-null  object 
 10  accidents           45920 non-null  float64
 11  owners              45918 non-null  float64
 12  make                91325 non-null  object 
 13  model               91311 non-null  object 
 14  trim                76174 non-null  object 
 15  bodyStyle           91310 non-null  object 
 16  year

In [238]:
# print descriptive stats of some cols
print(df_combined["price_kbbValue"].describe())

print(df_combined["price_total"].describe())
print(df_combined["milesPerGallon"].describe())


count     91327.000000
mean      24183.254555
std       13932.024279
min        5453.000000
25%       14562.828897
50%       21842.312370
75%       30211.948594
max      125069.000000
Name: price_kbbValue, dtype: float64
count     91301.000000
mean      25261.284729
std       14833.087543
min        1950.000000
25%       17499.000000
50%       22342.000000
75%       29590.000000
max      499998.000000
Name: price_total, dtype: float64
count    86730.000000
mean        29.274369
std         19.397878
min          0.000000
25%         21.500000
50%         25.000000
75%         30.000000
max        150.000000
Name: milesPerGallon, dtype: float64


In [209]:
#print(df_combined[df_combined["milesPerGallon"] > 50])
#print(df_combined.head(5))
# milesPerGallon for electric must be ignored

This needs to be repeated for complete dataset

In [302]:
df_combined["state"].value_counts()
#More than 6 states available.
# 87,291 are from 6 states of interest

state
Florida           22467
California        17898
Texas             14583
Oregon            12816
Massachusetts     11109
Georgia            8418
Washington          920
Connecticut         773
South Carolina      378
New Hampshire       355
Alabama             253
Rhode Island        241
New York            146
New Mexico           95
Illinois             94
Virginia             78
New Jersey           77
North Carolina       61
Nevada               55
Oklahoma             54
Colorado             54
Ohio                 49
Arizona              49
Tennessee            37
Indiana              36
Pennsylvania         36
Missouri             28
Maryland             20
Kansas               19
Michigan             18
Idaho                13
Utah                 13
Hawaii               12
Kentucky             11
Iowa                  9
Minnesota             9
Louisiana             7
Wisconsin             6
Vermont               6
Nebraska              5
Arkansas              5
Maine     

In [311]:
state_counts = df_combined["state"].value_counts()
selected_counts = state_counts.loc[selected_states]
print(selected_counts)

selected_states = ["California", "Florida", "Texas", "Massachusetts", "Georgia", "Oregon"]

# Count occurrences of each state and sum them
total_count = df_combined["state"].isin(selected_states).sum()
print(total_count)

state
California       17898
Florida          22467
Texas            14583
Massachusetts    11109
Georgia           8418
Oregon           12816
Name: count, dtype: int64
87291
